# Defining a Web API in a Notebook

In this notebook, we'll define a RESTful web API for adding, updating, removing, and listing the members of a contact list. We'll then deploy our notebook as a service using the Jupyter Kernel Gateway.

The contact list supported by this notebook is intentionally simple. See https://github.com/jupyter/kernel_gateway_demos for more complex examples and http://jupyter-kernel-gateway.readthedocs.io/en/latest/http-mode.html for full documentation.

**Table of Contents**

1. [Definition](#Definition)
2. [Implementation](#Implementation)
    1. [Create a contact](#Create-a-contact)
    2. [Update a contact](#Update-a-contact)
    3. [Delete a contact](#Delete-a-contact)
    4. [Get contacts](#Get-contacts)
3. [Deployment](#Deployment)
4. [Test](#Test)

## Definition

Let's start by outlining the resources for our contact list.

* `POST /contacts` &rarr; create a new contact
* `PUT /contacts/:contact_id` &rarr; update a contact
* `DELETE /contacts/:contact_id` &rarr; delete a contact
* `GET /contacts?name=<regex>` &rarr; get contacts, optionally filtered by name

We'll also state that requests and responses should carry JSON content for consistency.

## Implementation

In [1]:
import json
import os
import uuid

For simplicity, we'll use a global dictionary to store our contact list. If we later want to make our list persistent or our web service scale to multiple workers, we can switch to a true data store.

In [2]:
contacts = {}

These are the fields we'll allow for each contact.

In [3]:
fields = ['name', 'phone', 'address']

### Create a contact

We'll want to get the values for the name, phone, and address fields from the client when we create a contact. The kernel gateway will set the `REQUEST` variable to a JSON string containing information from the client. Let's synthesize an example request here in order to develop the code to add a contact to our contact list.

In [4]:
REQUEST = json.dumps({
    'body': {
        'name': 'Jane Doe',
        'phone': '888-555-5245',
        'address': '123 Bellview Drive, Somewhere, NC'
    }
})

Now let's write the handler code. We'll also annotate it so that the kernel gateway knows the code in this cell should execute when a client sends a HTTP `POST` request to the `/contacts` path.

In [5]:
# POST /contacts
# decode the request
req = json.loads(REQUEST)
# pull out the body
body = req['body']
# generate a new contact ID
new_contact_id = str(uuid.uuid4())
# put what we can about the contact in the dictionary
contacts[new_contact_id] = {field: body.get(field) for field in fields}
print(json.dumps({'contact_id': new_contact_id}))

{"contact_id": "9217e734-4dfd-4c32-a58a-375794bffb7e"}


We can print the contacts to see if it contains the data from our sample request, and see that it does.

In [6]:
contacts

{'9217e734-4dfd-4c32-a58a-375794bffb7e': {'address': '123 Bellview Drive, Somewhere, NC',
  'name': 'Jane Doe',
  'phone': '888-555-5245'}}

It's worth pointing out that we can put development code like this in our notebook without harming how it works with the kernel gateway as long as our notebook can run top to bottom. With a little more effort, we can even write basic tests in our notebook that run only when we're authoring or editing the notebook.

### Update a contact

We can follow the same pattern to implement updates to existing contacts. We start with an example request. We'll use the `new_contact_id` we generated above, but we'll be careful not to assume that it exists.

In [7]:
REQUEST = json.dumps({
    'body': {
        'name': 'Jane and John Doe',
        'address': '321 Viewbell Lane, Somewhere Else, SC'
    },
    'path': {
        'contact_id': globals().get('new_contact_id', '')
    }
})

This time, we need to know the identity assigned to the contact we're updating. We'll get that by declaring a variable in the path of the resource, `contact_id`, and read the value from the request.

In [8]:
# PUT /contacts/:contact_id
req = json.loads(REQUEST)
body = req['body']
contact_id = req['path']['contact_id']
if contact_id in contacts:
    contacts[contact_id].update({field: body[field] for field in fields if field in body})
    status = 200
    print(json.dumps({'contact_id': contacts[contact_id]}))
else:
    status = 404

{"contact_id": {"name": "Jane and John Doe", "phone": "888-555-5245", "address": "321 Viewbell Lane, Somewhere Else, SC"}}


In [9]:
contacts

{'9217e734-4dfd-4c32-a58a-375794bffb7e': {'address': '321 Viewbell Lane, Somewhere Else, SC',
  'name': 'Jane and John Doe',
  'phone': '888-555-5245'}}

Notice how we set `status` to `200` when we have the requested contact and `404` when we don't. These are the HTTP status codes we want to return to help clients distinguish successful requests from failures. We can annotate a separate cell with `ResponseInfo` and print this status code, [as well as other information](http://jupyter-kernel-gateway.readthedocs.io/en/latest/http-mode.html#setting-the-response-status-and-headers) such as the mimetype of the response, in a JSON blob. The kernel gateway will parse this info and use it in the response to the client's HTTP request.

In [10]:
# ResponseInfo PUT /contacts/:contact_id
print(json.dumps({
    "status" : status,
    "headers" : {
        "Content-Type" : "application/json"
    }
}))

{"headers": {"Content-Type": "application/json"}, "status": 200}


### Delete a contact

Our deletion code is much the same. We take a contact ID and remove it from our dict if it exists. We respond with a reasonable status code for successful deletion or a failure when the given ID is not found.

In [11]:
# DELETE /contacts/:contact_id
req = json.loads(REQUEST)
contact_id = req['path']['contact_id']
if contact_id in contacts:
    del contacts[contact_id]
    # HTTP status code for no body
    status = 204
else:
    # HTTP status code for not found
    status = 404

In [12]:
# ResponseInfo DELETE /contacts/:contact_id
print(json.dumps({
    "status" : status,
    
}))

{"status": 204}


### Get contacts

Finally, our code to fetch contacts must support a query parameter according to our [resource definition](#Definition). We'll implement a function that filters contacts by name given an optional regular expression.

In [13]:
import re

In [14]:
def filter_by_name(name_regex, contacts):
    """Get contacts with names matching the optional regex.
    
    Get all contacts if name_regex is None.
    
    Parameters
    ----------
    name_regex: str or None
        Regular expression to match to contact names
    contacts: list of dict
        Contacts to consider
        
    Returns
    -------
    list of dict
        Matching contacts
    """
    if name_regex is not None:
        return {contact_id: contact for contact_id, contact in contacts.items()
                if re.search(name_regex, contact['name'], re.IGNORECASE)}
    else:
        return contacts

We can get the regular expression from query portion of the request URL.

In [15]:
# GET /contacts
req = json.loads(REQUEST)
# query args appear as a list since they can be repeated in the URL
name_regex = req.get('args', {}).get('name', [None])[0]
hits = filter_by_name(name_regex, contacts)
print(json.dumps(hits))

{}


In [16]:
# ResponseInfo GET /contacts
print(json.dumps({
    "headers" : {
        "Content-Type" : "application/json"
    }
}))

{"headers": {"Content-Type": "application/json"}}


## Deployment

Now we're ready to deploy our notebook-defined API using the kernel gateway. We'll run a command like the following in a shell session to start the server:

```bash
python kernel_gateway --api='kernel_gateway.notebook_http' \
    --seed_uri='etc/api_examples/api_intro.ipynb' \
    --port 8889
```

We need to adjust the `seed_uri` to point to the location of this notebook on disk either using an absolute path or a path relative to where we run the command.

## Test

We can send a few requests to our new server to see how it responds. We'll put the client code in the same notebook as the API definition to keep the tutorial contained in a single notebook.  We'll make sure the client code doesn't run when the kernel gateway starts by checking if the `KERNEL_GATEWAY` environment variable is defined or not.

In [ ]:
URL = 'http://127.0.0.1:8889'

In [ ]:
if 'KERNEL_GATEWAY' not in os.environ:
    import requests

    # create a contact
    post_resp = requests.post(URL+'/contacts', json={
        'name': 'Alice Adams',
        'phone': '919-555-6712',
        'address': '42 Wallaby Way, Sydney, NC'
    })
    post_resp.raise_for_status()
    print('created a contact:', post_resp.json())
    
    first_contact_id = post_resp.json()['contact_id']

    # update the contact
    put_resp = requests.put(URL+'/contacts/'+first_contact_id, {
        'phone': '919-444-5601'
    })                      
    put_resp.raise_for_status()
    print('\nupdated a contact:', put_resp.json())

    # add two more contacts
    requests.post(URL+'/contacts', json={
        'name': 'Bob Billiham',
        'phone': '860-555-1409',
        'address': '3712 Not Real Lane, Bridgeport, CT'
    }).raise_for_status()
    requests.post(URL+'/contacts', json={
        'name': 'Cathy Caritgan',
        'phone': '512-555-6925',
        'address': '11 Stringigent Road, Albany, NY'
    }).raise_for_status()
    print('\added two more contacts')

    # fetch contacts with 'billi' in the lowercased text
    resp = requests.get(URL+'/contacts?name=billi')
    resp.raise_for_status()
    print('\ncontacts w/ name Bill:', resp.json())

    # delete a contact
    requests.delete(URL+'/contacts/'+first_contact_id).raise_for_status()
    print('\ndeleted a contact')
    
    # show all of the remaining contacts
    resp = requests.get(URL+'/contacts')
    resp.raise_for_status()
    print('\nall contacts:', resp.json())